In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar el archivo CSV
df = pd.read_csv('facturas.csv')

In [3]:
# Asegurarse de que la columna 'Factura_FechaHora' sea de tipo datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'], errors='coerce', format='%d/%m/%Y')

# solo tomar en cuenata Emp_Id = 1
df = df[df['Emp_Id'] == 1]

# Seleccionar las características relevantes
features = ['Factura_FechaHora', 'Suc_Id', 'Caja_Id', 'TipoDoc_Id', 'Factura_Estado', 'Factura_Fecha']
df = df[features]

# Crear nuevas características a partir de 'Factura_FechaHora'
df['Hora'] = df['Factura_FechaHora'].dt.hour
df['Dia_Semana'] = df['Factura_Fecha'].dt.dayofweek  + 1

# Eliminar la columna 'Factura_FechaHora' y 'Factura_Fecha'
df = df.drop(columns=['Factura_FechaHora'])

# Calcular la cantidad de transacciones que un empleado puede manejar en una hora
transacciones_por_hora = 42 / 4.8 

# Agrupar por sucursal, día de la semana y hora para obtener el número de transacciones
df_grouped = df.groupby(['Suc_Id', 'Dia_Semana', 'Hora', 'Factura_Fecha']).size().reset_index(name='Cantidad_Transacciones')

# Calcular el número de cajas activas por sucursal
cajas_activas = df.groupby(['Suc_Id', 'Caja_Id']).size().reset_index().groupby('Suc_Id').size().reset_index(name='Cajas_Activas')

# Unir el DataFrame de cajas activas con el DataFrame agrupado
df_grouped = df_grouped.merge(cajas_activas, on='Suc_Id')


# Mostrar el DataFrame agrupado
print(df_grouped)

C:\Users\angel\AppData\Local\Temp\ipykernel_12424\289794131.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')


        Suc_Id  Dia_Semana  Hora Factura_Fecha  Cantidad_Transacciones  \
0            1           1     7    2024-10-21                       5   
1            1           1     7    2024-10-28                       3   
2            1           1     8    2024-10-07                       2   
3            1           1     8    2024-10-14                       1   
4            1           1     8    2024-10-21                       1   
...        ...         ...   ...           ...                     ...   
118244     328           7    18    2024-10-27                       1   
118245     328           7    19    2024-10-13                       2   
118246     328           7    19    2024-10-20                       1   
118247     328           7    20    2024-10-13                       2   
118248     328           7    20    2024-10-20                       3   

        Cajas_Activas  
0                   2  
1                   2  
2                   2  
3              

In [64]:
# que dias de la semana hay en el dataset
print(df_grouped['Dia_Semana'].unique())

[1 2 3 4 5 6 7]


### Personal necesario
Este dato sera en base a la cantidad de transacciones por hora

In [ ]:
# Calcular la cantidad de personal necesario basado en las transacciones esperadas
df_grouped['Personal_Necesario'] = df_grouped['Cantidad_Transacciones'] / transacciones_por_hora
df_grouped['Personal_Necesario'] = np.ceil(df_grouped['Personal_Necesario'])  # Redondear hacia arriba
print(df_grouped)


        Suc_Id  Dia_Semana  Hora Factura_Fecha  Cantidad_Transacciones  \
0            1           1     7    2024-10-21                       5   
1            1           1     7    2024-10-28                       3   
2            1           1     8    2024-10-07                       2   
3            1           1     8    2024-10-14                       1   
4            1           1     8    2024-10-21                       1   
...        ...         ...   ...           ...                     ...   
118244     328           7    18    2024-10-27                       1   
118245     328           7    19    2024-10-13                       2   
118246     328           7    19    2024-10-20                       1   
118247     328           7    20    2024-10-13                       2   
118248     328           7    20    2024-10-20                       3   

        Cajas_Activas  Personal_Necesario  
0                   2                 1.0  
1                   2  

In [73]:



# sacar una fila del dia martes de la sucursal 3 y darme un json de las ventas por hora de esta forma simila: transacciones_por_hora = {
#     0: 10, 1: 5, 2: 3, 3: 2, 4: 1, 5: 1, 6: 2, 7: 12, 8: 4, 9: 15, 10: 14, 11: 12,
#     12: 3, 13: 11, 14: 23, 15: 1, 16: 31, 17: 12, 18: 31, 19: 12, 20: 33, 21: 12, 22: 12, 23: 5
# } Factura_Fecha  = 10 de octubre de 2024
df_grouped_day = df_grouped[(df_grouped['Suc_Id'] == 3) & (df_grouped['Factura_Fecha'] == '2024-10-10')]

# guardar en un json data.json
df_grouped_day.to_json('data.json', orient='records')

transacciones_por_hora = df_grouped_day.groupby('Hora')['Cantidad_Transacciones'].sum()
transacciones_por_hora = transacciones_por_hora.to_dict()

print(transacciones_por_hora)

{0: 14, 1: 4, 2: 1, 3: 3, 4: 3, 5: 3, 6: 4, 7: 13, 8: 6, 9: 10, 10: 6, 11: 13, 12: 19, 13: 14, 14: 10, 15: 8, 16: 4, 17: 8, 18: 11, 19: 18, 20: 3, 21: 12, 22: 8, 23: 7}


In [74]:
# para la sucursal 3, cual es el numero maximo de transacciones que se han realizado en una hora el dia de semana 4
df_grouped_max = df_grouped[(df_grouped['Suc_Id'] == 3) & (df_grouped['Dia_Semana'] == 4)]
max_transacciones = df_grouped_max['Cantidad_Transacciones'].max()
print(max_transacciones)

22
